Code to retrieve multi-domain proteins for analysis of domain to domain predicted aligned error.

In [1]:
import requests
import pandas as pd
import numpy as np
import random
import csv

In [3]:
'''
Build the PDB Search API request
'''

# Set the search parameters
url = 'https://search.rcsb.org/rcsbsearch/v2/query'
query = {
  "query": {
    "type": "group",
    "nodes": [
      {
        "type": "group",
        "logical_operator": "and",
        "nodes": [
          {
            "type": "terminal",
            "service": "text",
            "parameters": {
              "attribute": "rcsb_polymer_instance_annotation.annotation_lineage.id",
              "operator": "exact_match",
              "value": "56572",
              "negation": False
            }
          },
          {
            "type": "terminal",
            "service": "text",
            "parameters": {
              "attribute": "rcsb_polymer_instance_annotation.type",
              "operator": "exact_match",
              "value": "SCOP",
              "negation": False
            }
          }
        ],
        "label": "nested-attribute"
      },
    ],
    "logical_operator": "and",
    "label": "text"
  },
  "return_type": "polymer_entity",
  "request_options": {
    "group_by_return_type": "groups",
    "group_by": {
      "aggregation_method": "matching_uniprot_accession",
      "ranking_criteria_type": {
        "sort_by": "rcsb_entry_info.resolution_combined",
        "direction": "asc"
      }
    },
    "return_all_hits": True,
    "results_content_type": [
      "experimental"
    ],
    "sort": [
      {
        "sort_by": "score",
        "direction": "desc"
      },
      {
        "sort_by": "size",
        "direction": "desc"
      }
    ],
    "scoring_strategy": "combined"
  }
}

In [4]:
response = requests.post(url, json=query)

response_dic = response.json()

# Get the PDB IDs
group_set = response_dic['group_set']

# Get the list of uniprot ids
uniprots = [group_set[i]['identifier'] for i in range(len(group_set))]


# with open('./project_pipeline/data/multi_domain_uniprots.csv', 'w') as f:
#     writer = csv.writer(f)
#     writer.writerow(['uniprot'])
#     writer.writerows(uniprots)


In [5]:
'''
Commented out
'''
# def get_domains(uniprot_id):
#     '''
#     Get the domain information from UniProtKB
#     '''
#     print(f'Getting domains for {uniprot_id}')
#     url = f'https://rest.uniprot.org/uniprotkb/search?query=accession:{uniprot_id}&fields=ft_domain'
#     response = requests.get(url)
#     response_dic = response.json()
#     domains = []
#     try:
#         features = response_dic['results'][0]['features']
#         # Get the start and end of any domains
#         for i in range(len(features)):
#             if response_dic['results'][0]['features'][0]['type'] == 'Domain':
#                 start = str(features[i]['location']['start']['value'])
#                 end = str(features[i]['location']['end']['value'])
#                 domains.append((start + '-' + end))

#         domains_string = ','.join(domains)

#     except KeyError:
#         print(f'No domains found for {uniprot_id}')
#         domains_string = None

#     return domains_string

# def single_domains(uniprots):
#     '''
#     Get the single domains from UniProtKB
#     '''
#     domains = {'uniprot': [], 'region': []}
#     # Get domains for the uniprot ids
#     for i in range(len(uniprots)):
#         uniprot_id = uniprots[i]
#         region = get_domains(uniprot_id)
#         domains['uniprot'].append(uniprot_id)
#         domains['region'].append(region)

#     # Convert to pandas dataframe
#     domains_df = pd.DataFrame.from_dict(domains, orient='columns')

#     return domains_df

# domains_df = single_domains(uniprots)

Getting domains for P03366
Getting domains for P33334
Getting domains for P04585
Getting domains for P12497
Getting domains for P04050
Getting domains for P08518
Getting domains for P03300
Getting domains for P03303
Getting domains for P03367
Getting domains for Q97W02
Getting domains for Q38087
Getting domains for P00811
Getting domains for P26663
Getting domains for Q8RQE9
Getting domains for Q8RQE8
Getting domains for P62593
Getting domains for P19821
Getting domains for Q99ZW2


KeyboardInterrupt: 

In [6]:
'''
Commented out
'''
# domains_df = domains_df.drop(domains_df[domains_df['region'] == ''].index).reset_index(drop=True)
# domains_df = domains_df.dropna().reset_index(drop=True)

# # Remove any proteins with only one annotated domain
# for i in range(len(domains_df)):
#     region = domains_df.loc[i, 'region']
#     count = region.count('-')
#     if count <= 1:
#         domains_df = domains_df.drop(i)

# # remove any proteins that may overlap with my autoinhibited set
# autoinhibited = pd.read_csv('./project_pipeline/data/classified_files_3.tsv', sep='\t').astype('object')
# common = domains_df['uniprot'].isin(autoinhibited['uniprot'])
# print(common)
# domains_df = domains_df.drop(domains_df[common].index).reset_index(drop=True)

# # Save the dataframe
# domains_df.to_csv('./project_pipeline/data/multi_domain_domains.csv', index=False)

NameError: name 'domains_df' is not defined

In [2]:
autoinhibited = pd.read_csv('./project_pipeline/data/classified_files_3.tsv', sep='\t').astype('object')
single_df = pd.read_csv('./project_pipeline/data/single_domain_domains.csv')
common2 = single_df['uniprot'].isin(autoinhibited['uniprot'])

In [3]:
'''
How many multi-domain proteins have only two designated domains?
'''
md = pd.read_csv('./project_pipeline/data/multi_domain_domains.csv')

# Get the number of domains
md['num_domains'] = md['region'].str.count(',') + 1

# Get the number of proteins with only two domains
md_2 = md[md['num_domains'] == 2]
print(len(md_2))

61


In [3]:
'''
Select proteins with two domains. Label those domains 'region 1' and 'region 2' and pass the protein
through my predicted aligned error script.
'''
domains_df = pd.read_csv('./project_pipeline/data/multi_domain_domains.csv').astype('object')

domains_df['region'] = domains_df['region'].str.split(',')
two_domains_list = [row for index, row in domains_df.iterrows() if len(row['region']) == 2]
two_domains = pd.DataFrame(two_domains_list).reset_index(drop=True)

expand = pd.DataFrame(two_domains['region'].to_list(), columns=['region_1', 'region_2'])
expand = expand.join(two_domains['uniprot'], how='left')
expand = expand[['uniprot', 'region_1', 'region_2']]
expand.head()

expand.to_csv('./project_pipeline/data/multi_domain_regions.csv', index=False)